In [20]:
import pandas as pd
file = pd.read_csv('../data/cleaned_complete.csv').drop(['Unnamed: 0','Unnamed: 0.1','index','dateUpdated','dateListed','geoLocation','geometry', 'index_right', 'SA2_maincode_2021','Population Increase','Earners (persons)','Median age of earners (years)','Male','Female',	'Person','Male Ratio','Female Ratio','Sum ($)','Mean ($)','Median ($)'],axis = 1)

In [160]:
population = file[['Population ','surburb','year']].rename({'surburb':'suburb'},axis = 1)

In [161]:
house = file.groupby(['bathrooms', 'bedrooms', 'carspaces','surburb','year','month'],as_index = False)['rent','Population '].median('rent').rename({'surburb':'suburb','Population ':'population'},axis = 1)

/var/folders/rx/_11wjxvj4hx57z74nvgkbvpr0000gn/T/ipykernel_19247/804941803.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  house = file.groupby(['bathrooms', 'bedrooms', 'carspaces','surburb','year','month'],as_index = False)['rent','Population '].median('rent').rename({'surburb':'suburb','Population ':'population'},axis = 1)


In [168]:
house = house.set_index(['month','year'])

In [170]:
cpi_combined_df = pd.read_csv('../data/cpi_df.csv')
cpi_combined_df = cpi_combined_df.set_index(['month','year'])
cpi_combined_df 

,,Consumer_price_index
month,year,
8,2018,1.9
9,2018,1.9
10,2018,1.9
11,2018,1.8
12,2018,1.8
...,...,...
8,2024,3.5
9,2024,3.5
10,2024,3.0


In [172]:
#个数不同
merged_data = house.join(cpi_combined_df, on = ['month','year'],how = 'left').reset_index()
merged_data

,month,year,bathrooms,bedrooms,carspaces,suburb,rent,population,Consumer_price_index
0,5,2022,0.0,1.0,0.0,carlton,175.0,16701.220181,6.1
1,8,2022,0.0,1.0,0.0,clyde north,200.0,24822.102194,6.1
2,12,2019,0.0,1.0,0.0,docklands,850.0,16375.000000,1.8
3,3,2022,0.0,1.0,0.0,ivanhoe,300.0,12678.988491,5.1
4,10,2019,0.0,1.0,0.0,newport,250.0,19243.000000,1.7
...,...,...,...,...,...,...,...,...,...
51423,11,2021,4.0,5.0,4.0,craigieburn,540.0,22005.000000,3.5
51424,6,2022,4.0,5.0,4.0,rockbank,650.0,25309.447163,6.1
51425,8,2019,4.0,5.0,4.0,traralgon,570.0,16489.000000,1.7
51426,1,2019,4.0,5.0,4.0,tyabb,800.0,18825.000000,1.8
